# John's Swing-Trading Code Python Translation

In [1]:
from datetime import date, timedelta
import pandas_market_calendars as mcal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from pathlib import Path
import random
import os
from google.cloud import bigquery
from google.oauth2 import service_account
import sys

In [2]:
gbq_proj_id = 'stock-chipper-87578'

def config(username=None):
    if username == 'reese':
        file_dir = "C:/Users/rsmcd/OneDrive/Desktop/Trade Review/stock-chipper-research/"
        credential_file = 'stock-chipper-87578-ec8b427fca6a.json'
    elif username == 'ben':
        file_dir = "C:/Users/benwo/Documents/repos/stock-chipper-app/creds/"
        credential_file = "stock-chipper-87578-b17ad3f7e6e1.json"
    else:
        raise ValueError(f"Unrecognized or missing username: {username}")
    
    return file_dir + credential_file

# Use default when in Jupyter or if __main__ but no args
try:
    if __name__ == "__main__":
        if len(sys.argv) <= 1:
            raise ValueError("No username provided via CLI args")
        private_key = config(sys.argv[1])
    else:
        private_key = config("reese")  # Default for notebook
except Exception as e:
    print("Falling back to default username (reese)")
    private_key = config("reese")

Falling back to default username (reese)


Translated from swing-functions.r

In [3]:
def get_most_recent_trading_day(current_date=None):
    if current_date is None:
        current_date = date.today()
        
    nyse = mcal.get_calendar('NYSE')
    schedule = nyse.valid_days(start_date=(current_date - timedelta(days=30)).isoformat(),
                               end_date=current_date.isoformat())
    valid_days = [d.date() for d in schedule]
    
    for i in range(1, 21):
        candidate = current_date - timedelta(days=i)
        if candidate in valid_days:
            return candidate
    raise ValueError("Could not find recent trading day.")

def get_ticker_data(ticker, date_pull_begin=None, local=False, credentials_path=None, project_id=None):
    if local:
        file_path = Path("swing-trading/local-ticker-data") / f"{ticker}.txt"
        return pd.read_csv(file_path, sep="\t", parse_dates=["date_time"])

    if date_pull_begin is None:
        date_pull_begin = pd.Timestamp.today() - pd.DateOffset(years=2)

    sql = f"""
        SELECT ticker, date_time, high, low
        FROM `stock-chipper-87578.first_rate.stocks_candle_1day`
        WHERE date_time > '{date_pull_begin:%Y-%m-%d}' AND ticker = '{ticker}'
        ORDER BY date_time ASC
    """

    return run_sql_query(sql, project_id=project_id, credentials_path=credentials_path)

def get_trades(data, upper_bound, lower_bound, time_start):
    state = 0  # 0 = not in position, 1 = in position
    trades = []

    for i, row in data.iterrows():
        if row["date_time"] < time_start:
            continue

        if state == 0 and row["low"] <= lower_bound:
            trades.append({
                "date": row["date_time"],
                "type": "buy",
                "daily_high": row["high"],
                "daily_low": row["low"],
                "trade_price": lower_bound
            })
            state = 1
        elif state == 1 and row["high"] >= upper_bound:
            trades.append({
                "date": row["date_time"],
                "type": "sell",
                "daily_high": row["high"],
                "daily_low": row["low"],
                "trade_price": upper_bound
            })
            state = 0

    if state == 1:  # Open position still held; force close
        last = data.iloc[-1]
        trades.append({
            "date": last["date_time"],
            "type": "sell",
            "daily_high": last["high"],
            "daily_low": last["low"],
            "trade_price": 0.5 * (last["high"] + last["low"])
        })

    return pd.DataFrame(trades)

def get_returns(data, upper_bound, lower_bound, time_start, starting_cash=10000):
    trades = get_trades(data, upper_bound, lower_bound, time_start)

    if trades.empty:
        return {"annualized_return": None, "num_trades": 0}

    shares = 0
    cash = starting_cash

    for _, trade in trades.iterrows():
        if trade["type"] == "buy":
            shares = cash / trade["trade_price"]
            cash = 0
        elif trade["type"] == "sell":
            cash = shares * trade["trade_price"]
            shares = 0

    last_day = data["date_time"].max()
    period_years = (last_day - pd.to_datetime(time_start)).days / 365.25

    if period_years == 0:
        return {"annualized_return": None, "num_trades": trades['type'].eq("buy").sum()}

    total_return = cash / starting_cash - 1
    annualized = (cash / starting_cash) ** (1 / period_years) - 1

    return {"annualized_return": annualized, "num_trades": trades['type'].eq("buy").sum()}

def analyze_ticker_data(data, grid_size=20, num_pse=1.5):
    """
    Optimize buy/sell bounds for swing trading strategy.

    Args:
        data (pd.DataFrame): Daily OHLC data with 'low', 'high', 'date_time'
        grid_size (int): Number of thresholds to test per axis
        num_pse (float): Penalization factor on return

    Returns:
        pd.DataFrame: Single-row DataFrame with optimal lb, ub, return, etc.
    """
    lb_start = data['low'].quantile(0.01)
    lb_end = data['low'].quantile(0.75)

    ub_start = data['high'].quantile(0.10)
    ub_end = data['high'].quantile(0.99)

    time_start = data['date_time'].min()

    lb_values = np.linspace(lb_start, lb_end, grid_size)
    ub_values = np.linspace(ub_start, ub_end, grid_size)

    experiments = []

    for lb in lb_values:
        for ub in ub_values:
            if lb >= ub:
                continue
            result = get_returns(data, upper_bound=ub, lower_bound=lb, time_start=time_start)
            annualized_return = result["annualized_return"]
            num_trades = result["num_trades"]

            spread = ub - lb
            pseudo_se = abs(annualized_return) / np.sqrt(num_trades) if num_trades > 0 else np.nan
            return_lb = annualized_return - num_pse * pseudo_se if num_trades > 0 else np.nan

            experiments.append({
                "lb": lb,
                "ub": ub,
                "spread": spread,
                "return": annualized_return,
                "trades": num_trades,
                "pseudo_se": pseudo_se,
                "return_lb": return_lb,
                "time_start": time_start
            })

    df = pd.DataFrame(experiments).dropna(subset=["return_lb"])

    if df.empty:
        raise ValueError("No valid parameter combinations found.")

    max_return_lb = df["return_lb"].max()

    # Choose best result with tiebreak on max trades and smallest spread
    best = df[df["return_lb"] == max_return_lb].sort_values(["trades", "spread"], ascending=[False, True]).tail(1)

    return best.reset_index(drop=True)

def evaluate_recent_performance(data, month_lookback=3):
    """
    Compute weighted average of recent evaluation returns.
    More recent months get higher weight.

    Args:
        data (pd.DataFrame): Should include 'evaluation_period_end' and 'evaluation_return'
        month_lookback (int): Number of months to look back from most recent period

    Returns:
        float: Weighted average evaluation return
    """
    data = data.copy()
    data["evaluation_period_end"] = pd.to_datetime(data["evaluation_period_end"])

    last_period = data["evaluation_period_end"].max()
    cutoff = last_period - pd.DateOffset(months=month_lookback)

    recent = data[
        (data["evaluation_period_end"] >= cutoff) &
        (data["evaluation_period_end"] <= last_period)
    ][["evaluation_period_end", "evaluation_return"]].copy()

    if recent.empty:
        return 0.0

    # Treat NA returns as zero
    recent["evaluation_return"] = recent["evaluation_return"].fillna(0.0)

    # Compute months_ago weight (1 / (months_ago + 1))
    recent["months_ago"] = ((last_period - recent["evaluation_period_end"]).dt.days / 30) + 1
    recent["wt"] = 1 / recent["months_ago"]
    recent["wt"] = recent["wt"] / recent["wt"].sum()

    recent["wtd_returns"] = recent["wt"] * recent["evaluation_return"]

    return recent["wtd_returns"].sum()

def plot_swing_trades(result_row, local=False):
    """
    Plot the price action, trade points, and evaluation/analysis periods.

    Args:
        result_row (pd.Series or single-row DataFrame): One row from ticker_results
        local (bool): Whether to load data from local file
    """
    if isinstance(result_row, pd.DataFrame):
        result_row = result_row.iloc[0]

    overall_start = pd.to_datetime(result_row["analysis_period_start"])
    overall_end = pd.to_datetime(result_row["evaluation_period_end"])
    analysis_end = pd.to_datetime(result_row["analysis_period_end"])
    eval_start = pd.to_datetime(result_row["evaluation_period_start"])

    ticker = result_row["ticker"]
    buy_level = result_row["analysis_buy"]
    sell_level = result_row["analysis_sell"]

    # Pull data and filter
    ticker_data = get_ticker_data(ticker, overall_start, local=local)
    ticker_data = ticker_data[ticker_data["date_time"] <= overall_end]

    # Get trades from both periods
    trades_analysis = get_trades(
        ticker_data[
            (ticker_data["date_time"] >= result_row["analysis_period_start"]) &
            (ticker_data["date_time"] <= result_row["analysis_period_end"])
        ],
        upper_bound=sell_level,
        lower_bound=buy_level,
        time_start=result_row["analysis_period_start"]
    )

    trades_eval = get_trades(
        ticker_data[
            (ticker_data["date_time"] >= result_row["evaluation_period_start"]) &
            (ticker_data["date_time"] <= result_row["evaluation_period_end"])
        ],
        upper_bound=sell_level,
        lower_bound=buy_level,
        time_start=result_row["evaluation_period_start"]
    )

    trades = pd.concat([trades_analysis, trades_eval], ignore_index=True)

    # Add trade price as the bound unless out of range, in which case midpoint
    trades["price"] = trades.apply(
        lambda row: row["trade_price"]
        if row["daily_low"] <= row["trade_price"] <= row["daily_high"]
        else 0.5 * (row["daily_high"] + row["daily_low"]),
        axis=1
    )

    # Plot setup
    fig, ax = plt.subplots(figsize=(14, 6))

    # Error bars (high/low ranges)
    ax.errorbar(ticker_data["date_time"], 
                y=(ticker_data["high"] + ticker_data["low"]) / 2,
                yerr=(ticker_data["high"] - ticker_data["low"]) / 2,
                fmt='-', ecolor='gray', alpha=0.4)

    # Horizontal lines for buy/sell levels
    ax.axhline(buy_level, color="palegreen", linestyle="--", label="Buy Level")
    ax.axhline(sell_level, color="lightcoral", linestyle="--", label="Sell Level")

    # Vertical line for analysis period end
    ax.axvline(analysis_end, color="gray", linestyle=":", label="Analysis Ends")
    ax.text(analysis_end, ticker_data["low"].min(), "Analysis Ends", rotation=90, va="bottom", ha="right", color="gray")

    # Plot trade points
    buy_trades = trades[trades["type"] == "buy"]
    sell_trades = trades[trades["type"] == "sell"]
    ax.scatter(buy_trades["date"], buy_trades["price"], color="green", label="Buy", zorder=5)
    ax.scatter(sell_trades["date"], sell_trades["price"], color="red", label="Sell", zorder=5)

    # Title and caption
    caption = (
        f"Analysis Trades: {len(trades_analysis[trades_analysis['type'] == 'sell'])}\n"
        f"Analysis Return: {result_row['analysis_return']:.2%}\n"
        f"Evaluation Trades: {len(trades_eval[trades_eval['type'] == 'sell'])}\n"
        f"Evaluation Return: {result_row['evaluation_return']:.2%}\n"
        f"Full Range: {overall_start.date()} to {overall_end.date()}"
    )

    ax.set_title(f"Swing Trades for {ticker}", fontsize=14)
    ax.set_xlabel("")
    ax.set_ylabel("Price")
    ax.legend()
    ax.text(0.01, 0.01, caption, transform=ax.transAxes, fontsize=10,
            va='bottom', ha='left', bbox=dict(facecolor='white', alpha=0.5))

    # Format date axis
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))

    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def analyze_ticker(
    ticker,
    month_offset=9,
    date_pull_begin=None,
    length_out=15,
    verbose_output=True,
    save_experiment=False,
    date_slug=None
):
    """
    Analyze one ticker using swing strategy optimization.

    Args:
        ticker (str): Ticker symbol
        month_offset (int): Months to look back for optimization
        date_pull_begin (str or datetime): Override for data start
        length_out (int): Number of thresholds in grid
        verbose_output (bool): Whether to print plots and tables
        save_experiment (bool): Save grid results to file
        date_slug (str): Optional filename prefix

    Returns:
        pd.DataFrame: Top grid result(s)
    """
    if not month_offset and not date_pull_begin:
        raise ValueError("You must provide either month_offset or date_pull_begin")

    if date_pull_begin is None:
        date_pull_begin = pd.Timestamp.today() - pd.DateOffset(months=int(1.25 * month_offset))
    else:
        date_pull_begin = pd.to_datetime(date_pull_begin)

    # Pull data
    data = get_ticker_data(ticker, date_pull_begin)

    lb_start = data["low"].quantile(0.01)
    lb_end = data["low"].quantile(0.75)
    ub_start = data["high"].quantile(0.10)
    ub_end = data["high"].quantile(0.99)

    last_date = data["date_time"].max()
    time_start = last_date - pd.DateOffset(months=month_offset)

    # Build grid
    lb_vals = np.linspace(lb_start, lb_end, length_out)
    ub_vals = np.linspace(ub_start, ub_end, length_out)
    grid = []

    for lb in lb_vals:
        for ub in ub_vals:
            if lb >= ub:
                continue
            result = get_returns(data, upper_bound=ub, lower_bound=lb, time_start=time_start)
            grid.append({
                "lb": lb,
                "ub": ub,
                "time_start": time_start,
                "return": result["annualized_return"],
                "trades": result["num_trades"]
            })

    grid_df = pd.DataFrame(grid).dropna(subset=["return"])

    if save_experiment:
        if not date_slug:
            date_slug = pd.Timestamp.today().strftime("%Y%m%d")
        output_file = Path("swing-trading/experiments") / f"{date_slug}_{ticker}_swing_experiments.txt"
        grid_df.to_csv(output_file, sep="\t", index=False)

    if grid_df.empty:
        print("No valid trade setups found.")
        return grid_df

    max_trades = grid_df["trades"].max()
    returns_80pct = grid_df["return"].quantile(0.80)

    if max_trades <= 6:
        top = grid_df.sort_values(["trades", "return"], ascending=[True, True]).tail(5 if verbose_output else 1)
    else:
        top = grid_df[(grid_df["return"] > returns_80pct)].sort_values("trades").tail(5 if verbose_output else 1)

    if verbose_output and not top.empty:
        best = top.iloc[-1]
        best_trades = get_trades(data, best["ub"], best["lb"], best["time_start"])
        print(best_trades[["date", "type", "trade_price"]].to_string(index=False))
        print("---------------------------")

        # Plot
        plot_data = data[data["date_time"] >= best["time_start"]]
        fig, ax = plt.subplots(figsize=(14, 6))
        ax.errorbar(
            plot_data["date_time"],
            y=(plot_data["high"] + plot_data["low"]) / 2,
            yerr=(plot_data["high"] - plot_data["low"]) / 2,
            fmt='-', ecolor='gray', alpha=0.5
        )
        ax.axhline(best["lb"], color="lightblue", linestyle="--")
        ax.axhline(best["ub"], color="lightblue", linestyle="--")
        ax.set_title(f"Swings for {ticker}")
        ax.set_ylabel("Price")
        ax.set_xlabel("")
        ax.text(0.01, 0.01, f"Maximum return: {best['return']:.1%}", transform=ax.transAxes,
                bbox=dict(facecolor='white', alpha=0.5))
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    return top.reset_index(drop=True)

def run_sql_query(sql, project_id, credentials_path=None):
    """
    Execute a BigQuery SQL query using service account credentials.

    Args:
        sql (str): The SQL query to run.
        project_id (str): Your GCP project ID.
        credentials_path (str): Full path to your service account JSON key.

    Returns:
        pd.DataFrame: Query results as a DataFrame.
    """
    if credentials_path is None:
        raise ValueError("credentials_path must be provided")

    credentials = service_account.Credentials.from_service_account_file(credentials_path)
    client = bigquery.Client(credentials=credentials, project=project_id)

    query_job = client.query(sql)
    result = query_job.result()
    return result.to_dataframe()

Translated from swing-walk-forward.r

In [4]:
# Config Flags
get_ticker_list_from_gbq = True
use_local_data = False
build_results = True
fresh_start = True
num_to_test = -10  # If <0, use all
output_stub = "20240101_"

# Path setup
DATA_DIR = Path("strategies/swing-trading/data")
WALK_FORWARD_DIR = DATA_DIR / "walk-forward"
MINUTE_TICKERS_FILE = DATA_DIR / "minute-tickers.txt"

DATA_DIR.mkdir(parents=True, exist_ok=True)
WALK_FORWARD_DIR.mkdir(parents=True, exist_ok=True)

# Pull tickers from DB or local file
if get_ticker_list_from_gbq:
    # You'll need to implement this function to return a DataFrame
    sql = '''
        SELECT DISTINCT ticker 
        FROM main.tf_stocks_for_backtesting("2023-08-01","2024-01-01")
    '''
    all_tickers = run_sql_query(sql, project_id=gbq_proj_id, credentials_path=private_key) # placeholder
    all_tickers["random_index"] = random.sample(range(len(all_tickers)), len(all_tickers))
    all_tickers.to_csv(MINUTE_TICKERS_FILE, sep="\t", index=False)
else:
    all_tickers = pd.read_csv(MINUTE_TICKERS_FILE, sep="\t")

# Filter tickers
if num_to_test > 0:
    tickers_to_test = all_tickers[all_tickers["random_index"] <= num_to_test]["ticker"].tolist()
else:
    tickers_to_test = all_tickers["ticker"].tolist()

# fresh_start logic
if build_results:
    overall_finish = get_most_recent_trading_day()
    num_periods = 3
    analysis_period = timedelta(days=6*7)
    evaluation_period = timedelta(days=4*7)
    end_of_last_analysis_period = overall_finish - evaluation_period
    overall_start = end_of_last_analysis_period - num_periods * analysis_period

    expected_min_analysis_days = 6 * 4

    if fresh_start:
        for file in WALK_FORWARD_DIR.glob("*"):
            file.unlink()
    else:
        existing_files = [f.stem.replace(output_stub, "") for f in WALK_FORWARD_DIR.glob("*.txt")]
        tickers_to_test = [t for t in tickers_to_test if t not in existing_files]

from tqdm import tqdm  # optional: for a progress bar
import warnings
warnings.filterwarnings("ignore")  # silence any pandas SettingWithCopy warnings

start = datetime.now()

for ticker in tqdm(tickers_to_test, desc="Processing tickers"):

    # Pull daily data for this ticker
    daily_data = get_ticker_data(
        ticker,
        overall_start,
        local=use_local_data,
        credentials_path=private_key,
        project_id=gbq_proj_id
    )
    
    analysis_period_starts = [
        end_of_last_analysis_period - analysis_period * i
        for i in range(1, num_periods + 1)
    ]

    # Build base results table
    ticker_results = pd.DataFrame({
        "ticker": ticker,
        "analysis_period_start": pd.to_datetime(analysis_period_starts),
    })
    
    ticker_results["analysis_period_end"] = ticker_results["analysis_period_start"] + analysis_period
    ticker_results["analysis_buy"] = 0.0
    ticker_results["analysis_sell"] = 0.0
    ticker_results["analysis_return"] = 0.0
    ticker_results["analysis_trades"] = 0
    ticker_results["analysis_eval_metric"] = 0.0
    ticker_results["evaluation_period_start"] = ticker_results["analysis_period_end"] + timedelta(days=1)
    ticker_results["evaluation_period_end"] = ticker_results["evaluation_period_start"] + evaluation_period
    ticker_results["evaluation_return"] = 0.0
    ticker_results["evaluation_trades"] = 0
    ticker_results["evaluation_data_good"] = False

    for idx, row in ticker_results.iterrows():
        # Slice data for the analysis period
        analysis_data = daily_data[
            (daily_data["date_time"] >= row["analysis_period_start"]) &
            (daily_data["date_time"] <= row["analysis_period_end"])
        ]

        if len(analysis_data) < expected_min_analysis_days:
            continue

        evaluation_data = daily_data[
            (daily_data["date_time"] >= row["evaluation_period_start"]) &
            (daily_data["date_time"] <= row["evaluation_period_end"])
        ]

        if not analysis_data.empty:
            period_results = analyze_ticker_data(analysis_data)

            ticker_results.at[idx, "analysis_buy"] = period_results["lb"][0]
            ticker_results.at[idx, "analysis_sell"] = period_results["ub"][0]
            ticker_results.at[idx, "analysis_return"] = period_results["return"][0]
            ticker_results.at[idx, "analysis_trades"] = period_results["trades"][0]
            ticker_results.at[idx, "analysis_eval_metric"] = period_results["return_lb"][0]
        else:
            ticker_results.at[idx, "analysis_return"] = np.nan

        if not evaluation_data.empty:
            ticker_results.at[idx, "evaluation_data_good"] = True 

            eval_results = get_returns(
                evaluation_data,
                sell_threshold=ticker_results.at[idx, "analysis_sell"],
                buy_threshold=ticker_results.at[idx, "analysis_buy"],
                start_time=ticker_results.at[idx, "evaluation_period_start"]
            )

            ticker_results.at[idx, "evaluation_trades"] = eval_results["num_trades"]
            ticker_results.at[idx, "evaluation_return"] = eval_results["annualized_return"]

    # Save result to file
    ticker_output_file = f"{output_stub}{ticker}.txt"
    output_path = WALK_FORWARD_DIR / ticker_output_file
    ticker_results.to_csv(output_path, sep="\t", index=False)

    print(f"Completed {ticker}")
    recent_results = evaluate_recent_performance(ticker_results)
    print(f"Wtd Avg of recent eval returns: {recent_results:.2%}")

else:
    results = "need to implement"

finish = datetime.now()
print("Total runtime:", finish - start)

c:\Users\rsmcd\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
Processing tickers:   0%|          | 1/4487 [00:01<1:52:42,  1.51s/it]

Completed GLBZ
Wtd Avg of recent eval returns: 0.00%


Processing tickers:   0%|          | 2/4487 [00:02<1:43:24,  1.38s/it]

Completed TIPT
Wtd Avg of recent eval returns: 0.00%


Processing tickers:   0%|          | 3/4487 [00:04<1:46:15,  1.42s/it]

...

Completed MP
Wtd Avg of recent eval returns: 0.00%


Processing tickers: 100%|█████████▉| 4486/4487 [1:28:21<00:01,  1.22s/it]

Completed DQ
Wtd Avg of recent eval returns: 0.00%


Processing tickers: 100%|██████████| 4487/4487 [1:28:22<00:00,  1.18s/it]

Completed TARO
Wtd Avg of recent eval returns: 0.00%
Total runtime: 1:28:22.402053
